# Sprachidentifikation mit fasttext
Dieses Notebook demonstriert, wie einfach man ein gutes LID-Modell mit fastText trainieren kann.
Wir können mit dem fasttext Befehlszeilen-Werkzeug oder mit der Python-Bibliothek arbeiten.
Aus Effizienzgründen arbeiten wir hier mit einem kleineren Trainingsdatensatz.

Conda-Umgebung mit FastText bereitstellen im Terminal und dann Jupyter Notebook nochmals starten.

```bash
conda activate base
conda create --name lid --clone base
conda activate lid
conda install -c conda-forge fasttext 
# danach neues Terminal öffnen und conda aktivieren

# fasttext Python Modul installieren
pip install fasttext

# danach Jupyter neu starten
jupyter notebook
```

In [ ]:
! which fasttext

Datenset runterladen

In [1]:
! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs21/casdmit/impresso-lid/impresso-lid.tsv -o impresso-lids.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  382k  100  382k    0     0  2147k      0 --:--:-- --:--:-- --:--:-- 2135k:-- 1953k


Datenset kennenlernen

In [2]:
! head -n 10 impresso-lid.tsv

__label__fr	Phoenix Bergban ... . . —it Siemens Halske . . . V» Ve Internat. Bank Ltuemb. — _ _ Deutsche Bank.. . . . . . °, Dresdncr Bank . . . . .— y« Discontc-Gesellschaft: . .— . so/* Berliner Handelsges. . . . \“la Rdchsbank. .- .
__label__lb	Den englesche plang fir Palestina Den englesche plang fir Palestina get den arabeschen delegatio'nen an der jüddescher agence virgeluegt. No gewessen informatio'nen soll dese plang follgend ponkten emfâssen: . Schafong vun cantonen. e jüddeschen an en arabeschen. . Dreiparteien-centralregirong zu Jerusalem. . Drei dietricter zu Jerusalem, e jüddeschen, en arabeschen an e klengeren fir 'Araber müss
__label__de	oie »or regeln formte ; nur müßte unfer SKilttärtocfen »oüfiänbiger unb foftjpteliger toerben unb mügten »ir benÄönig »on al« oberften Ärieg«berrn aners ïenr.en. Bir geben ju, bag ba« Sanb bei ginfüb= rung ber beutfdjen $anbel«gefeggebung, be« beutfdjen tyofc unb ïelegrapbenœefen« «. f. »., toie lrt. be« SJÎeformprojefte« fagt, getttnnen

In [3]:
! wc impresso-lid.tsv

  1000  60519 391400 impresso-lid.tsv


In [4]:
!  cut -f 1 < impresso-lid.tsv | sort | uniq -c 

    577 __label__de
     33 __label__en
    306 __label__fr
     27 __label__it
     57 __label__lb


Erstellen von Training und Testdaten (Originaldaten sind zufällig geordnet)

In [5]:
! head -n 900 < impresso-lid.tsv > impresso-lid-train.tsv
! tail -n 100 < impresso-lid.tsv > impresso-lid-test.tsv

## Trainieren von Minimodell auf der Kommandozeile
Erstellt die binäre Modelldatei impresso-lid.bin und die Textdatei impresso-lid.bin.vec, mit den Wörtern in der ersten Spalte und den 10-dimensionalen Vektor in den restlichen Spalten.
Falls das Kommandozeilenwerkzeug fasttext nicht installiert werden kann, siehe unten für Python-Interface


In [6]:
! fasttext supervised -input impresso-lid-train.tsv -output impresso-lid  \
  -minn 2 -maxn 2 -epoch 2 -dim 10 -bucket 2000 -lr 0.1

Read 0M words
Number of words:  25553
Number of labels: 5
Progress: 100.0%  words/sec/thread: 2079738  lr: 0.000000  loss: 1.553597  eta: 0h0m 


In [7]:
! tail impresso-lid.bin.vec

E.>A. -0.032914 -0.0016127 -0.0010814 0.021594 -0.066347 -0.0097248 -0.028648 0.01295 -0.044182 0.013695 
Rhein, -0.014979 0.0019735 0.010733 -0.014917 -0.040745 -0.041285 0.0082548 0.028197 -0.029087 0.027753 
E..U, -0.015263 0.032085 0.02272 0.0019828 0.00076419 -0.027352 -0.0086408 0.01078 -0.019888 0.0095381 
z?j -0.0016278 0.019604 -0.024708 -0.024287 0.009169 -0.0027684 0.034293 0.011582 -0.04343 0.054782 
Ludw,-Befb. -0.018409 0.01405 0.015361 0.008566 0.0057746 -0.0050782 -0.0060172 0.015789 -0.028111 0.025906 
E,.U. 0.0022321 0.025047 0.0031072 0.033288 0.010047 0.0091517 -0.0026759 0.013451 -0.030512 0.019293 
Franz,.Osteir. 0.018798 0.0014039 -0.00177 0.0087512 -0.038055 0.015682 -0.013943 0.013332 -0.027953 0.019872 
E..U. -0.027672 0.025487 -0.0024582 0.019228 -0.0017994 0.0014774 -0.026825 0.00082545 -0.036302 0.0049875 
Frcntfuit -0.0084553 -0.0044217 0.019875 0.018095 -0.045599 0.016002 0.0085338 0.026802 -0.020121 0.0042561 
M.. -0.0067887 0.030199 -0.010099 -0.0085391

## Evaluieren von Minimodell

In [8]:
! fasttext test impresso-lid.bin impresso-lid-test.tsv

N	100
P@1	0.59
R@1	0.59
Number of examples: 100


## Verbessern des Modells
Verbessern des Modells: Z.B. mehr Epochen, mehr Dimensionen, längere Buchstaben-N-Gramme, ...

Wichtigste Parameter:
```
   -epoch N  # Beim Lernen wird das ganze Trainingsset N mal benutzt.
   -dim N    # Länge der gelernten Vektoren für Wörter und Buchstaben-N-Gramme
   -bucket N # Maximale Anzahl verschiedener Merkmale, die repräsentierbar sind. Falls es mehr Merkmale als Platz im "bucket", überlagern sich die Merkmale.
   -lr 0.N   # Initiale Lernrate: Bestimmt, wie stark die Vektoren verändert werden, wenn Fehler passieren. Während des Lernens wird die Lernrate immer kleiner.
   -mmin N   # Minimale Länge der Subwords, d.h. Buchstaben-N-Gramme
   -maxn N   # Maximale Länger der Subwords, d.h. Buchstaben-N-Gramme (falls N=0, werden keine Subwords benutzt, nur Wörter)
   -minCount N # Minimales Vorkommen von Wörtern, um einbezogen zu werden
```

In [27]:
! fasttext supervised -input impresso-lid-train.tsv -output impresso-lid  \
  -minn 2 -maxn 2 -epoch 100 -dim 20 -bucket 20000 -lr 0.5 -minCount 0
! fasttext test impresso-lid.bin impresso-lid-test.tsv

Read 0M words
Number of words:  25553
Number of labels: 5
Progress: 100.0%  words/sec/thread: 1765579  lr: 0.000000  loss: 0.010311  eta: 0h0m ords/sec/thread: 1789978  lr: 0.459592  loss: 0.201550  eta: 0h0m 1774104  lr: 0.232200  loss: 0.017781  eta: 0h0m 1771734  lr: 0.091878  loss: 0.012066  eta: 0h0m 0.061393  loss: 0.010959  eta: 0h0m 
N	100
P@1	0.97
R@1	0.97
Number of examples: 100


Verwechslungskandidaten anzeigen

In [11]:
! cut -f 1 < impresso-lid-test.tsv > impresso-lid-test.gold.label.tsv
! fasttext predict impresso-lid.bin impresso-lid-test.tsv > impresso-lid-test.system.label.tsv
! paste impresso-lid-test.gold.label.tsv impresso-lid-test.system.label.tsv | sort | uniq -c | sort -rn

     59 __label__de	__label__de
     26 __label__fr	__label__fr
      6 __label__lb	__label__de
      5 __label__it	__label__fr
      2 __label__fr	__label__de
      2 __label__en	__label__fr


## Trainieren von FastText aus Python
Siehe https://fasttext.cc/docs/en/python-module.html

In [12]:
import fasttext

In [13]:
model = fasttext.train_supervised(input='impresso-lid-train.tsv', minn=2,maxn=2,epoch=20,dim=10,bucket=2000,lr=0.1)

In [14]:
model.get_subwords('Sprache')

(['Sprache', '<S', 'Sp', 'pr', 'ra', 'ac', 'ch', 'he', 'e>'],
 array([ 7243, 26969, 26119, 25592, 26093, 27118, 26831, 26807, 26043]))

In [15]:
print(model.labels)

['__label__de', '__label__fr', '__label__lb', '__label__en', '__label__it']


In [16]:
model.predict("Welche Sprache ist das? Luxemburgisch?",k=5)

(('__label__fr', '__label__de', '__label__lb', '__label__en', '__label__it'),
 array([0.5468235 , 0.23518349, 0.09703483, 0.06793454, 0.05307357]))

In [17]:
model.test("impresso-lid-test.tsv")

(100, 0.85, 0.85)

In [18]:
def print_results(N, p, r):
    print(f"N\t{N}")
    print(f"P@1\t{p:.2f}")
    print(f"R@1\t{r:.2f}")

In [19]:
print_results(*model.test("impresso-lid-test.tsv"))

N	100
P@1	0.85
R@1	0.85


# LID Shared-Task
Arbeitet in Zweiergruppen in Breakout-Rooms und versucht, ein besseres Modell zu trainieren. Tragt die beste Konfiguration im Spreadsheet https://cutt.ly/casdmit-fs21-lid ein.

 - Reflexion: Welche Hyperparameter scheinen keinen wesentlichen Einfluss zu haben? Welche den grössten Einfluss?